In [ ]:
# !pip install split-folders

In [1]:
import sys
sys.path.insert(1, '/home/as15798/.local/lib/python3.8/site-packages')

In [2]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [3]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/home/as15798/.kaggle’: File exists


In [4]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download hereisburak/pins-face-recognition

pins-face-recognition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
# !unzip /content/pins-face-recognition.zip

In [7]:
import splitfolders
# splitfolders.ratio('/content/105_classes_pins_dataset', output="/content/data", seed=1337, ratio=(.999999, 0.000001))
splitfolders.fixed('./105_classes_pins_dataset', output="./data", seed=1337, fixed=(1, 1))

Copying files: 17534 files [06:26, 45.40 files/s]


In [8]:
# !pip install facenet_pytorch
# ! pip install tensorboard

In [45]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [46]:
data_dir = './105_classes_pins_dataset'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

In [47]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [48]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [49]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

Batch 549 of 549

In [50]:
# Incorporating Image Transformations
import os
from PIL import Image
import time
base_directory = './105_classes_pins_dataset_cropped/'

for celeb in os.listdir(base_directory):
    for image in os.listdir(base_directory+celeb):
        try:
            img = Image.open(base_directory+celeb+'/'+image)
            #transformed_img = transforms.functional.adjust_brightness(img, brightness_factor=1.25)
#             transformed_img = transforms.functional.adjust_contrast(img, contrast_factor=2.0)
            transformed_img = transforms.functional.adjust_saturation(img, saturation_factor=2.0)
            transformed_img.save(base_directory+celeb+'/'+image)
        except:
            print('error with this: ', base_directory+celeb+'/'+image)

error with this:  ./105_classes_pins_dataset_cropped/pins_Richard Harmon/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_Alex Lawther/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_Zendaya/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_Amanda Crew/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_Miley Cyrus/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_Robert Downey Jr/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_Katharine Mcphee/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_Selena Gomez/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_alycia dabnem carey/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_Alvaro Morte/.ipynb_checkpoints
error with this:  ./105_classes_pins_dataset_cropped/pins_Alexandra Daddario/.ipynb_checkp

In [51]:
time.sleep(100)

In [52]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [53]:
# # Incorporating Random Erasing
# from __future__ import absolute_import

# from torchvision.transforms import *

# from PIL import Image
# import random
# import math
# import numpy as np
# import torch

# class RandomErasing(object):
#   '''
#   Class that performs Random Erasing in Random Erasing Data Augmentation by Zhong et al. 
#   -------------------------------------------------------------------------------------
#   probability: The probability that the operation will be performed.
#   sl: min erasing area
#   sh: max erasing area
#   r1: min aspect ratio
#   mean: erasing value
#     -------------------------------------------------------------------------------------
#     '''
#   def __init__(self, probability = 0.5, sl = 0.02, sh = 0.4, r1 = 0.3, mean=[0.4914, 0.4822, 0.4465]):
#     self.probability = probability
#     self.mean = mean
#     self.sl = sl
#     self.sh = sh
#     self.r1 = r1
       
#   def __call__(self, img):
#     print('Here in Random')
#     if random.uniform(0, 1) > self.probability:
#       return img

#     for attempt in range(100):
#       print('Here in random loop')
#       area = img.size()[1] * img.size()[2]
  
#       target_area = random.uniform(self.sl, self.sh) * area
#       aspect_ratio = random.uniform(self.r1, 1/self.r1)

#       h = int(round(math.sqrt(target_area * aspect_ratio)))
#       w = int(round(math.sqrt(target_area / aspect_ratio)))

#       if w < img.size()[2] and h < img.size()[1]:
#           x1 = random.randint(0, img.size()[1] - h)
#           y1 = random.randint(0, img.size()[2] - w)
#           if img.size()[0] == 3:
#               img[0, x1:x1+h, y1:y1+w] = self.mean[0]
#               img[1, x1:x1+h, y1:y1+w] = self.mean[1]
#               img[2, x1:x1+h, y1:y1+w] = self.mean[2]
#           else:
#               img[0, x1:x1+h, y1:y1+w] = self.mean[0]
#           return img

#       return img

In [54]:
# %load transformsown.py
# !python3 '/content/transformsown.py'
# import /content/transforms_own
# import sys
# sys.path.insert(0, '/content/transformsown.py')

In [55]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization,
    transforms.RandomHorizontalFlip(),
    transforms.RandomErasing(p=0.4, scale=(0.02, 0.13), ratio=(0.3, 3.3), value='random', inplace=False),
#     transforms.ColorJitter(brightness=(1.0, 1.5), contrast=0, saturation=0, hue=0)
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [56]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [57]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10
epochs = 50
print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |   111/111  | loss:    4.8192 | fps: 1221.7794 | acc:    0.0068   

Epoch 1/50
----------
Train |   441/441  | loss:    2.8729 | fps:  338.2750 | acc:    0.2808   
Valid |   111/111  | loss:    2.2841 | fps: 1236.1765 | acc:    0.4012   

Epoch 2/50
----------
Train |   441/441  | loss:    1.6186 | fps:  338.1496 | acc:    0.5573   
Valid |   111/111  | loss:    1.4761 | fps: 1239.6202 | acc:    0.5977   

Epoch 3/50
----------
Train |   441/441  | loss:    1.1319 | fps:  338.0604 | acc:    0.6828   
Valid |   111/111  | loss:    1.1187 | fps: 1239.4438 | acc:    0.6872   

Epoch 4/50
----------
Train |   441/441  | loss:    0.8439 | fps:  338.0861 | acc:    0.7621   
Valid |   111/111  | loss:    0.8916 | fps: 1238.4919 | acc:    0.7508   

Epoch 5/50
----------
Train |   441/441  | loss:    0.6977 | fps:  338.0510 | acc:    0.7989   
Valid |   111/111  | loss:    0.9823 | fps: 1234.9357 | acc:    0.7328   

Epoch 6/50
----------
Train |   441/441  | loss: 

In [58]:
torch.save(resnet.state_dict(), "./model.pt")
torch.save(resnet, "./model_data.pt")

In [59]:
resnet.load_state_dict(torch.load('./model.pt'))
resnet.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [20]:
# from PIL import Image
# mtcnn = MTCNN(
#     image_size=160, margin=0, min_face_size=20,
#     thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
#     device=device
# )
# img = Image.open('/content/bill_gates.jpg')

# # Get cropped and prewhitened image tensor
# img_cropped = mtcnn(img, save_path='/content/bill_gates_crop.jpg')

# # Calculate embedding (unsqueeze to add batch dimension)
# img_embedding = resnet(img_cropped.unsqueeze(0)).detach()


In [21]:
# img_embedding

In [22]:
#  torch.max(img_embedding, dim = 1)

In [23]:
# saved_data = torch.load('./model_data.pt') # loading data.pt file
# embedding_list = saved_data[0] # getting embedding data
# name_list = saved_data[1] # getting list of names
# dist_list = [] # list of matched distances, minimum distance is used to identify the person

# for idx, emb_db in enumerate(embedding_list):
#     dist = torch.dist(img_embedding, emb_db).item()
#     dist_list.append(dist)
    
# idx_min = dist_list.index(min(dist_list))
# # return (name_list[idx_min], min(dist_list))

In [24]:
!pip install facenet_pytorch
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Running on device: cuda:0


In [25]:
from facenet_pytorch import MTCNN, InceptionResnetV1
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [26]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [27]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('data/test')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [28]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

/home/as15798/.local/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Face detected with probability: 0.999988
Face detected with probability: 0.998462
Face detected with probability: 1.000000
Face detected with probability: 0.999740
Face detected with probability: 0.999991
Face detected with probability: 0.999923
Face detected with probability: 0.999991
Face detected with probability: 0.999840
Face detected with probability: 0.999838
Face detected with probability: 0.999999
Face detected with probability: 0.999928
Face detected with probability: 0.999194
Face detected with probability: 0.999669
Face detected with probability: 1.000000
Face detected with probability: 0.999995
Face detected with probability: 0.999993
Face detected with probability: 0.999982
Face detected with probability: 0.999944
Face detected with probability: 0.997699
Face detected with probability: 1.000000
Face detected with probability: 0.999943
Face detected with probability: 0.999832
Face detected with probability: 0.999944
Face detected with probability: 0.998260
Face detected wi

In [29]:
# !zip -r /content/data/test/test_data.zip /content/

In [30]:
# torch.cuda.memory_summary(device=None, abbreviated=False)
# torch.cuda.empty_cache()

In [31]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [32]:
import pandas as pd
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
pd.DataFrame(dists, columns=names, index=names)

,pins_Adriana Lima,pins_Alex Lawther,pins_Alexandra Daddario,pins_Alvaro Morte,pins_Amanda Crew,pins_Andy Samberg,pins_Anne Hathaway,pins_Anthony Mackie,pins_Avril Lavigne,pins_Ben Affleck,...,pins_ellen page,pins_elon musk,pins_gal gadot,pins_grant gustin,pins_jeff bezos,pins_kiernen shipka,pins_margot robbie,pins_melissa fumero,pins_scarlett johansson,pins_tom ellis
pins_Adriana Lima,0.000000,1.325599,1.329934,1.246458,1.279254,1.443431,1.411525,1.395144,1.458632,1.336855,...,1.480190,1.507732,1.369224,1.394099,1.569103,1.266549,1.228655,1.302131,1.193268,1.380662
pins_Alex Lawther,1.325599,0.000000,1.402205,1.240859,1.432997,1.353384,1.348110,1.376177,1.391660,1.301451,...,1.256260,0.957482,1.459367,1.183750,1.227957,1.379494,1.389089,1.207791,1.476318,1.334805
pins_Alexandra Daddario,1.329934,1.402205,0.000000,1.386864,1.294815,1.344781,1.096827,1.339537,1.264138,1.429411,...,1.535436,1.357954,1.349859,1.518811,1.468063,1.383783,1.247738,1.500995,1.226920,1.359606
pins_Alvaro Morte,1.246458,1.240859,1.386864,0.000000,1.347627,1.296153,1.325413,1.481072,1.508568,1.148431,...,1.512362,1.263350,1.512572,1.055070,1.277457,1.432771,1.441041,1.375903,1.223220,1.351032
pins_Amanda Crew,1.279254,1.432997,1.294815,1.347627,0.000000,1.196947,1.364740,1.480915,1.356666,1.248150,...,1.449114,1.293645,1.434334,1.461101,1.434242,1.681915,1.474191,1.346133,1.391081,1.250174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pins_kiernen shipka,1.266549,1.379494,1.383783,1.432771,1.681915,1.602860,1.205690,1.322637,1.436092,1.522857,...,1.287662,1.437241,1.201461,1.301602,1.365374,0.000000,1.224632,1.285309,1.350731,1.397952
pins_margot robbie,1.228655,1.389089,1.247738,1.441041,1.474191,1.413459,1.114394,1.266875,1.316254,1.460588,...,1.455460,1.404711,1.139252,1.441463,1.382796,1.224632,0.000000,1.458740,1.335582,1.432266
pins_melissa fumero,1.302131,1.207791,1.500995,1.375903,1.346133,1.343643,1.509942,1.435084,1.498568,1.390729,...,0.912126,1.444513,1.285529,1.361962,1.442754,1.285309,1.458740,0.000000,1.486662,1.247858
pins_scarlett johansson,1.193268,1.476318,1.226920,1.223220,1.391081,1.565991,1.319737,1.318206,1.351572,1.300651,...,1.376545,1.420243,1.267631,1.334486,1.330676,1.350731,1.335582,1.486662,0.000000,1.454244
